In [1]:
!pip install datasets

In [2]:
from transformers import MarianMTModel, MarianTokenizer, T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset
import torch


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from transformers import MarianMTModel, MarianTokenizer
model_save_path = '/content/drive/My Drive/MUN/Courses/ENGI-9867/Helsinki-NLP_trained_model.pt'
helsinki_model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-fr')
helsinki_model.load_state_dict(torch.load(model_save_path))
tokenizer1 = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-fr')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [5]:
model_save_path = '/content/drive/My Drive/MUN/Courses/ENGI-9867/t5_small_trained_model.pt'
t5_model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer2 = T5Tokenizer.from_pretrained('t5-small')

# Load the saved state dictionary into the model
t5_model.load_state_dict(torch.load(model_save_path))


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<All keys matched successfully>

In [13]:
def translate_with_models(text, model1, tokenizer1, model2, tokenizer2):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    def translate(text, model, tokenizer):
        # Move model to the appropriate device
        model.to(device)

        if model.config.name_or_path.startswith('Helsinki-NLP'):
            inputs = tokenizer(text, return_tensors='pt', padding=True)
        else:
            inputs = tokenizer('translate English to French: ' + text, return_tensors='pt', padding=True)

        # Move inputs to the appropriate device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=50)

        translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return translated_text

    translated_text1 = translate(text, model1, tokenizer1)
    translated_text2 = translate(text, model2, tokenizer2)

    return translated_text1, translated_text2


In [14]:
def transate_comparison(text):
  translation1, translation2 = translate_with_models(text, helsinki_model, tokenizer1, t5_model, tokenizer2)
  print("helsinki translation:\t", translation1)
  print("T5 translation:\t\t", translation2)


In [15]:
transate_comparison("Hello, how are you?")

helsinki translation:	 Bonjour, comment allez-vous?
T5 translation:		 Bonjour, comment êtes-vous?


In [16]:
transate_comparison("I want to eat a hamburger")

helsinki translation:	 Je veux manger un hamburger.
T5 translation:		 Je veux manger un hamburger


In [17]:
transate_comparison("I am happy")

helsinki translation:	 Je suis heureux.
T5 translation:		 Je suis heureuse


In [18]:
transate_comparison("I am sad")

helsinki translation:	 Je suis triste.
T5 translation:		 Je suis triste


In [19]:
transate_comparison("There should be difference between these two translations")

helsinki translation:	 Il devrait y avoir une différence entre ces deux traductions
T5 translation:		 Il faudrait y avoir une différence entre ces deux traductions.
